In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import keras.utils as image
from sklearn.preprocessing import OneHotEncoder
from keras.src.utils import to_categorical,normalize
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
import keras.layers as layers
from keras.models import Model

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
for dirname, _, filenames in os.walk("/kaggle/input/dataset-nn23/dataset"):
    Image_Count = 0
#     print(dirname)

    for file in filenames:
        Image_Count += 1

    if Image_Count > 0:
        print('Total Files in directory {} is {}'.format(dirname, Image_Count))

Total Files in directory /kaggle/input/dataset-nn23/dataset/test is 100
Total Files in directory /kaggle/input/dataset-nn23/dataset/train/2 is 1980
Total Files in directory /kaggle/input/dataset-nn23/dataset/train/5 is 1980
Total Files in directory /kaggle/input/dataset-nn23/dataset/train/3 is 1980
Total Files in directory /kaggle/input/dataset-nn23/dataset/train/1 is 1980
Total Files in directory /kaggle/input/dataset-nn23/dataset/train/4 is 1980


In [4]:
train_path = '/kaggle/input/dataset-nn23/dataset/train/'
test_path = '/kaggle/input/dataset-nn23/dataset/test/'

## Encoding ##

In [5]:
IMG_SIZE = 275
classes = ['1', '2', '3', '4', '5']
Input_Shape = (IMG_SIZE, IMG_SIZE, 3)
images = []
labels = []
for c in classes:
    for img in os.listdir(train_path + c):
        img = cv2.imread(train_path + c + '/' + img)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        images.append(img)
        labels.append(int(c))
        
images = np.array(images, dtype='float32')/255.0        
#images = np.array(images, dtype='float32')
#images= normalize(images,axis=1)


In [6]:
onehotencoder = OneHotEncoder()
label = np.array(labels).reshape(-1,1)
labels = onehotencoder.fit_transform(label).toarray()
# from keras.src.utils import to_categorical
#labels = to_categorical(labels, num_classes=5)

In [7]:
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, shuffle=True, random_state=42)

In [8]:
print(X_train.shape)
print(y_train.shape)
print(type(y_val))
print(type(X_val))

(7920, 275, 275, 3)
(7920, 5)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [9]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_val = np.array(X_val)
y_val = np.array(y_val)

## Augmentation For Train ##

In [10]:
training_datagen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    fill_mode="nearest",
)

training_datagen.fit(X_train)
# training_datagen.flow(X_train,y_train,batch_size=32)

In [11]:
print(X_val.shape)
print(y_val.shape)

(1980, 275, 275, 3)
(1980, 5)


In [12]:
def entry_flow(inputs):

    x = layers.Conv2D(32, 3, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(64, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for size in [128, 256, 728]:
        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(size, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(size, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

        residual = layers.Conv2D(  # Project residual
            size, 1, strides=2, padding='same')(previous_block_activation)           
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    return x


def middle_flow(x, num_blocks=8):
  
    previous_block_activation = x

    for _ in range(num_blocks):
        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(728, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(728, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        
        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(728, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)
        
        
        x = layers.Activation('relu')(x)
        x = layers.SeparableConv2D(728, 3, padding='same')(x)
        x = layers.BatchNormalization()(x)

        x = layers.add([x, previous_block_activation])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    return x


def exit_flow(x, num_classes=5):
  
    previous_block_activation = x

    x = layers.Activation('relu')(x)
    x = layers.SeparableConv2D(728, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Activation('relu')(x)
    x = layers.SeparableConv2D(1024, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

    residual = layers.Conv2D(  # Project residual
      1024, 1, strides=2, padding='same')(previous_block_activation)
    x = layers.add([x, residual])  # Add back residual

    x = layers.SeparableConv2D(1536, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.SeparableConv2D(2048, 3, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.GlobalAveragePooling2D()(x)
    ses, activation='softmax')(x)

inputs = keras.Input(shape=(275, 275, 3))
outputs = exit_flow(middle_flow(entry_flow(inputs)))
model = keras.Model(inputs, outputs)

# Compiling the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Checking Model Summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 275, 275, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 138, 138, 32)         896       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 138, 138, 32)         128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 138, 138, 32)         0         ['batch_normalization[0][0

In [ ]:
history_incption=model.fit(training_datagen.flow(X_train, y_train, batch_size=40),
         validation_data=(X_val, y_val),epochs=112,steps_per_epoch=130)

Epoch 1/112
130/130 [==============================] - 299s 2s/step - loss: 0.4839 - accuracy: 0.3727 - val_loss: 0.5185 - val_accuracy: 0.2242
Epoch 2/112
130/130 [==============================] - 229s 2s/step - loss: 0.3913 - accuracy: 0.5058 - val_loss: 0.6109 - val_accuracy: 0.2076
Epoch 3/112
130/130 [==============================] - 229s 2s/step - loss: 0.3588 - accuracy: 0.5629 - val_loss: 0.5858 - val_accuracy: 0.2288
Epoch 4/112
130/130 [==============================] - 229s 2s/step - loss: 0.3498 - accuracy: 0.5910 - val_loss: 0.5602 - val_accuracy: 0.5889
Epoch 5/112
130/130 [==============================] - 229s 2s/step - loss: 0.3449 - accuracy: 0.5948 - val_loss: 0.3989 - val_accuracy: 0.5015
Epoch 6/112
130/130 [==============================] - 229s 2s/step - loss: 0.3335 - accuracy: 0.6196 - val_loss: 0.6514 - val_accuracy: 0.5561
Epoch 7/112
130/130 [==============================] - 229s 2s/step - loss: 0.3382 - accuracy: 0.6156 - val_loss: 0.8505 - val_accuracy:

In [ ]:
X_test = []
image_coulmn=[]
for img in os.listdir(test_path):
    image_coulmn.append(int((img.split('.'))[0]))
    img = cv2.imread(test_path + img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
   # img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    # Preprocess the image if needed (e.g., resizing, normalization)
    img = cv2.resize(img,(275, 275))  # Resize to match your target size
    X_test.append(img)

# Convert the list of images to a NumPy array
X_test = np.array(X_test)/255.0

In [ ]:
Output=model.predict(X_test, batch_size=10, verbose=0)

In [ ]:
Prediction_classes=[]
for predict in Output:
    index=np.argmax(predict)
    Predition_classes.append(index+1)

In [ ]:
Prediction_classes

In [ ]:
# Create a dictionary with two columns
data = {'image_id': image_coulmn,'label': Prediction_classes}

# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

df.to_csv('file6.csv',header=False,index=False)

###### 